In [ ]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('Coffee Shop Sales.csv', 'Coffee_Shop_Sales_2')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='hostname',
    user='username',
    password='your_password',
    database='db_name'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = r'CSV Path'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Prepare the INSERT statement
    insert_query = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(df.columns))})"

    # Batch insert using executemany
    values_to_insert = [tuple(None if pd.isna(x) else x for x in row) for row in df.values]
    cursor.executemany(insert_query, values_to_insert)

    # Commit the transaction for the current CSV file
    conn.commit()
    print(f"Inserted {len(values_to_insert)} records into {table_name}.")

# Close the connection
cursor.close()
conn.close()
